#  재순위화(Re-rank) 기법, 맥락 압축(Contextual Compression) 기법

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

`(4) 벡터스토어 로드`

In [ ]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

`(5) 백터 검색기 생성`

In [ ]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5}
)

query = "테슬라 트럭 모델이 있나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

---

## **Re-rank** (재순위화)

- **재순위화**는 검색 결과를 재분석하여 최적의 순서로 정렬하는 고도화된 기술임

- **이중 단계 프로세스**로 기본 검색 후 정교한 기준으로 재평가를 진행함
    1. 먼저 기본 검색 알고리즘으로 관련 문서들을 찾은 후, 
    2. 더 정교한 기준으로 이들을 재평가하여 최종 순위를 결정

- 사용자의 검색 의도에 맞는 **정확도 향상**을 통해 검색 품질을 개선함

- 검색 결과의 품질을 높이기 위한 체계적인 최적화 방법론

--- 
### 1) **Cross Encoder** Reranker

- **Cross-Encoder** 모델을 활용하여 검색 결과의 정밀한 재정렬을 수행함
- 데이터를 **쌍(pair) 단위**로 처리하여 문서와 쿼리 간의 관계를 분석함 (예: 두 개의 문장 또는 문서)
- **통합 인코딩 방식**으로 검색 쿼리와 검색된 문서 간 유사도를 더 정확하게 계산함

- 참고: https://www.sbert.net/examples/applications/cross-encoder/README.html

`(1) 모델 초기화`

- pip install hf_xet 또는 uv pip install hf_xet

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# CrossEncoderReranker 모델 초기화 
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# CrossEncoderReranker 모델을 사용한 re-ranker 초기화 (top_n: 3)
re_ranker = CrossEncoderReranker(model=model, top_n=3)

# CrossEncoderReranker를 사용한 retriever 초기화
cross_encoder_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=chroma_k_retriever,
)

`(2) 문서 검색`

In [ ]:
# CrossEncoderReranker를 사용한 retriever를 사용하여 검색
query = "테슬라 트럭 모델이 있나요?"
retrieved_docs = cross_encoder_reranker_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

### 2) **LLM** Reranker

- **대규모 언어 모델**을 활용하여 검색 결과의 재순위화를 수행함
- 쿼리와 문서 간의 **관련성 분석**을 통해 최적의 순서를 도출함
- **LLMListwiseRerank**와 같은 전문화된 재순위화 모델을 적용함

`(1) 모델 초기화`

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMListwiseRerank
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLMListwiseRerank 모델 초기화 (top_n: 3)
re_ranker = LLMListwiseRerank.from_llm(llm, top_n=3)

# LLMListwiseRerank 모델을 사용한 re-ranker 초기화
llm_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=chroma_k_retriever,
)

`(2) 문서 검색`

In [ ]:
# LLMListwiseRerank 모델을 사용한 retriever를 사용하여 검색

query = "테슬라 트럭 모델이 있나요?"
retrieved_docs = llm_reranker_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

---

## **Contextual Compression** (맥락적 압축)

- **맥락적 압축 기술**은 검색된 문서를 그대로 반환하는 대신, 쿼리 관련 정보만을 선별적으로 추출함

- **이중 구조 시스템**으로 기본 검색과 문서 압축 과정을 수행함
    1. 기본 검색기(base retriever) 
    2. 문서 압축기(Document Compressor)

- **효율적인 처리**를 통해 LLM 비용 절감과 응답 품질 향상을 달성함

### 1) **LLMChainFilter**

- **LLM 기반 필터링**으로 검색된 문서의 포함 여부를 결정함
- **원본 유지 방식**으로 문서 내용의 변경 없이 선별 작업을 수행함
- **선택적 필터링**을 통해 관련성 높은 문서만을 최종 반환함
- 문서 원본을 보존하면서 관련성 기반의 스마트한 선별을 수행하는 방식

`(1) 모델 초기화`

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLMChainFilter 모델 초기화
context_filter = LLMChainFilter.from_llm(llm)

# LLMChainFilter 모델을 사용한 retriever 초기화
llm_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=context_filter,                   # LLM 기반 압축기
    base_retriever=chroma_k_retriever,                # 기본 검색기 
)

`(2) 문서 검색`

In [ ]:
# LLMListwiseRerank 모델을 사용한 retriever를 사용하여 검색

query = "테슬라 트럭 모델이 있나요?"
compressed_docs = llm_filter_compression_retriever.invoke(query)

for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

### 2) **LLMChainExtractor**

- **LLM 기반 추출**로 문서에서 쿼리 관련 핵심 내용만을 선별함
- **순차적 처리 방식**으로 각 문서를 검토하여 관련 정보를 추출함
- **맞춤형 요약**을 통해 쿼리에 최적화된 압축 결과를 생성함
- 쿼리 맥락에 따른 선별적 정보 추출로 효율적인 문서 압축을 실현

`(1) 모델 초기화`

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLMChainExtractor 모델 초기화
compressor = LLMChainExtractor.from_llm(llm)

# LLMChainExtractor 모델을 사용한 retriever 초기화
llm_extractor_compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,                                    # LLM 기반 압축기
    base_retriever=cross_encoder_reranker_retriever,               # 기본 검색기 (Re-rank)
)

`(2) 문서 검색`

In [ ]:
# LLMChainExtractor 모델을 사용한 retriever를 사용하여 검색

query = "테슬라 트럭 모델이 있나요?"
compressed_docs = llm_extractor_compression_retriever.invoke(query)


for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

### 3) **EmbeddingsFilter**

- **임베딩 기반 필터링**으로 문서와 쿼리 간 유사도를 계산함
- **LLM 미사용 방식**으로 빠른 처리 속도와 비용 효율성을 확보함 (LLM 호출보다 저렴하고 빠른 옵션)
- **유사도 기준 선별**을 통해 관련성 높은 문서만을 효과적으로 추출함
- 경제적이고 신속한 임베딩 기반의 문서 필터링 기법 

`(1) 모델 초기화`

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# 임베딩 기반 압축기 초기화
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.4)

# 임베딩 기반 압축기를 사용한 retriever 초기화
embed_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,                             # 임베딩 기반 압축기
    base_retriever=cross_encoder_reranker_retriever,               # 기본 검색기 (Re-rank)
)

`(2) 문서 검색`

In [ ]:
query = "테슬라 트럭 모델이 있나요?"
compressed_docs = embed_filter_compression_retriever.invoke(query)

for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

### 4) **DocumentCompressorPipeline**

- **파이프라인 구조**로 여러 압축기를 순차적으로 연결하여 처리함
- **복합 변환 기능**으로 문서 분할 및 중복 제거 등 다양한 처리가 가능함
- **유연한 확장성**을 통해 BaseDocumentTransformers 추가로 기능을 확장함
- 다중 압축기를 연계하여 포괄적이고 효과적인 문서 처리를 구현하는 방식

`(1) 모델 초기화`

In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter


# 임베딩 기반 필터 초기화 - 중복 제거
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 임베딩 기반 필터 초기화 - 유사도 기반 필터 (임베딩 유사도 0.4 이상)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.4)

# Re-ranking 모델 초기화
re_ranker = LLMListwiseRerank.from_llm(llm, top_n=2)

# DocumentCompressorPipeline 초기화 (순차적으로 redundant_filter -> relevant_filter -> re_ranker 적용)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter, re_ranker]
)

# DocumentCompressorPipeline을 사용한 retriever 초기화
pipeline_compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,           # DocumentCompressorPipeline 기반 압축기
    base_retriever=chroma_k_retriever,             # 기본 검색기
)

`(2) 문서 검색`

In [ ]:
query = "테슬라 트럭 모델이 있나요?"
compressed_docs = pipeline_compression_retriever.invoke(query)

for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)